### Overview

blah blah.....


## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing

from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_log_error,r2_score,mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

## Load Dataset

In [ ]:
df=pd.read_csv('./data/vehicles.csv')

In [ ]:
df.head()

In [ ]:
df.info()

## Data Analysis


### Data Cleanup

##### Removing columns whihc are not needed

In [ ]:
df2=df.copy()
df2=df2.drop(columns=['url','region_url','VIN','image_url','description','county','state', 'posting_date'])
df2.head()

#### Null values handling

In [ ]:
df2.isnull().sum() 

In [ ]:
sns.heatmap(df2.isnull(),yticklabels=False,cbar=True,cmap='Accent')

In [ ]:
num_col=['year','odometer','lat','long']
cat_cols=['region','manufacturer','model','condition','cylinders','fuel','title_status','transmission','drive','size','type','paint_color']

#### Fill missing data

##### Numerical data

In [ ]:
num_col=['year','odometer','lat','long']

In [ ]:
df2.isnull().sum()[num_col]

In [ ]:
data_num=df2[num_col]
imputer_num=IterativeImputer( ExtraTreesRegressor(n_estimators=10, random_state=0)) #estimators[2] = ExtraTreesRegressor
impute_data=imputer_num.fit_transform(data_num)
df2[num_col]=impute_data

In [ ]:
df2.isnull().sum()[num_col]

##### Categorical Data


In [ ]:
cat_cols=['region','manufacturer','model','condition','cylinders','fuel','title_status','transmission','drive','size','type','paint_color']

In [ ]:
df2.isnull().sum()[cat_cols]

In [ ]:
data_cat=df2[cat_cols]
encoder=preprocessing.LabelEncoder()

In [ ]:
def encode(data):
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

In [ ]:
for columns in cat_cols:
    encode(data_cat[columns])
    imputer = IterativeImputer(BayesianRidge())
    impute_data=imputer.fit_transform(data_cat[columns].values.reshape(-1, 1))
    impute_data=impute_data.astype('int64')
    impute_data = pd.DataFrame(impute_data)
    impute_data =encoder.inverse_transform(impute_data.values.reshape(-1, 1))
    data_cat[columns]=impute_data
df2[cat_cols]=data_cat  

In [ ]:
df2.isnull().sum()[cat_cols]

##### Check missing values

In [ ]:
sns.heatmap(df2.isnull(),yticklabels=False,cbar=True,cmap='Accent')

In [ ]:
df2.isnull().sum()

##### Save the cleaned file

In [ ]:
df2.to_csv('./data/vehiclesData.csv',index=False)

#### Data Encoding and Normali

### Fix Outliers [TODO]

### Data Pre-processing

#### Encoding category fields

In [ ]:
le=preprocessing.LabelEncoder()
df2[cat_cols]=df2[cat_cols].apply(le.fit_transform)

In [ ]:
df2

#### Normalizing the data

In [ ]:
norm = StandardScaler()
df2['price'] = np.log(df2['price'])
df2['odometer'] = norm.fit_transform(np.array(df2['odometer']).reshape(-1,1))
df2['year'] = norm.fit_transform(np.array(df2['year']).reshape(-1,1))
df2['model'] = norm.fit_transform(np.array(df2['model']).reshape(-1,1))

# #scaling target variable
q1,q3=(df2['price'].quantile([0.25,0.75]))
o1=q1-1.5*(q3-q1)
o2=q3+1.5*(q3-q1)
df2=df2[(df2.price>=o1) & (df2.price<=o2)]

df2['region'] = norm.fit_transform(np.array(df2['region']).reshape(-1,1))
df2['lat'] = norm.fit_transform(np.array(df2['lat']).reshape(-1,1))
df2['long'] = norm.fit_transform(np.array(df2['long']).reshape(-1,1))

In [ ]:
df2.head()

## Builing ML Model

#### Util Methods

In [ ]:
def trainingData(df_loc,n):
    X = df_loc.iloc[:,n]
    y = df_loc.iloc[:,-1:].values.T
    y=y[0]
    X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.9,test_size=0.1,random_state=0)
    return (X_train,X_test,y_train,y_test)

X_train,X_test,y_train,y_test=trainingData(df2,list(range(len(list(df2.columns))-1)))

In [ ]:
def remove_neg(y_test,y_pred):
    ind=[index for index in range(len(y_pred)) if(y_pred[index]>0)]
    y_pred=y_pred[ind]
    y_test=y_test[ind]
    y_pred[y_pred<0]
    return (y_test,y_pred)

#function for evaluation of model
def result(y_test,y_pred):
    r=[]
    r.append(mean_squared_log_error(y_test, y_pred))
    r.append(np.sqrt(r[0]))
    r.append(r2_score(y_test,y_pred))
    r.append(round(r2_score(y_test,y_pred)*100,4))
    return (r)

#### Train-Test split

### Model Implementations

In [ ]:
#dataframe that store the performance of each model
accu=pd.DataFrame(index=['MSLE', 'Root MSLE', 'R2 Score','Accuracy(%)'])    

#### Linear Regression

In [ ]:
LR=LinearRegression()
LR.fit(X_train,y_train)
y_pred=LR.predict(X_test)

In [ ]:
y_test_1,y_pred_1=remove_neg(y_test,y_pred)

In [ ]:
r1_lr=result(y_test_1,y_pred_1)
print('Coefficients: \n', LR.coef_)
print("MSLE : {}".format(r1_lr[0]))
print("Root MSLE : {}".format(r1_lr[1]))
print("R2 Score : {} or {}%".format(r1_lr[2],r1_lr[3]))
accu['Linear Regression']=r1_lr


#### Random Forest

### Lasso Regression

#### K-Neighbors Regressor

### Vizuallizations [TODO]